### Template for generating code for scalable scientific computation

#### Fill in the project name and problem statement.

In [ ]:
project_name = ""
problem_statement = '''Replace this text with a description of the problem
you are trying to solve
'''

#### Set up the AI

In [ ]:
import json
from pathlib import Path

import gpt_engineer.steps as steps
from gpt_engineer.ai import AI, fallback_model
from gpt_engineer.db import DB, DBs

def set_up(project_name, problem_statement):
    input_path = Path(project_name)
    input_path.mkdir(parents=True, exist_ok=True)

    prompt_file = input_path / "prompt"

    with open(prompt_file, "w") as file:
        file.write(problem_statement)

    input_path = input_path.absolute()
    print("The following location will be used for processing\nThe code will be output to the workspace directory of that location")
    print(input_path)
    
    model = "gpt-4"
    temperature = 0.1
    model = fallback_model(model)
    ai = AI(
        model=model,
        temperature=temperature,
    )

    memory_path = input_path / "memory"
    workspace_path = input_path / "workspace"
    archive_path = input_path / "archive"

    dbs = DBs(
        memory=DB(memory_path),
        logs=DB(memory_path / "logs"),
        input=DB(input_path),
        workspace=DB(workspace_path),
        preprompts=DB(Path(steps.__file__).parent / "preprompts"),
        archive=DB(archive_path),
    )
    
    return ai, dbs

def do_step(step):
    messages = step(ai, dbs)
    dbs.logs[step.__name__] = json.dumps(messages)

In [ ]:
ai, dbs = set_up(project_name, problem_statement)

#### Let's do it

In [ ]:
do_step(steps.clarify)

In [ ]:
do_step(steps.gen_clarified_code)

In [ ]:
do_step(steps.gen_entrypoint)